In [2]:
from glob import glob
import os
import nibabel as nib
import numpy as np
import subprocess
import pandas as pd

# Volume Original

## 1 voxel = 0.12 mm (isotropic)

In [3]:
raw_df = pd.DataFrame()
raw_mris = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*_id.nii.gz')))
mapping_csv = "dataset3/GIN/labels_mapping.csv"
mapping_data = pd.read_csv(mapping_csv)

In [10]:
mask_map6s = sorted(glob(os.path.join('dataset3', 'GIN', 'Mask', 'M*_id.nii.gz')))
for mask_path in mask_map6s:
    name = mask_path.split('/')[-1].split('_')[0]
    #print(name)
    mask = nib.load(mask_path).get_fdata()
    totalbrain = (mask==1).sum()*0.12**3 
    #print(totalbrain)        
    new_row = {'name':name, 'raw_volume_brain':totalbrain}
    raw_df = raw_df.append(new_row, ignore_index=True)

/tmp/ipykernel_2563067/3385324798.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_df = raw_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2563067/3385324798.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_df = raw_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2563067/3385324798.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_df = raw_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2563067/3385324798.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_df = raw_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2563067/3385324798.py:9: FutureWarning: The frame.append method is deprecated and

In [11]:
raw_map6s = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', 'M*_id.nii.gz')))
labelvolumes = []
for j, labels_path in enumerate(raw_map6s):
    name = labels_path.split('/')[-1].split('_')[0]
    #print(name)
    labels = nib.load(labels_path).get_fdata()
    labelvolume = []
    for i in range(1,11):
        labelvolume.append((labels==i).sum()*0.12**3)
    labelvolumes.append(labelvolume)

for i in range(0,10):
    volume_k = []
    for j in range(len(raw_map6s)):
        volume_k.append(labelvolumes[j][mapping_data['map6nifti'][i]-1])
    raw_df['raw_volume_'+str(mapping_data['code'][i]).lower()] = volume_k

In [12]:
mask_svbps = sorted(glob(os.path.join('dataset3', 'GIN', 'Mask', '6*_id.nii.gz')))
for mask_path in mask_svbps:
    name = mask_path.split('/')[-1].split('_')[0]
    mask = nib.load(mask_path).get_fdata()
    totalbrain = (mask==1).sum()*0.12**3  
    raw_df.loc[len(raw_df)] = [name, totalbrain, None,None,None,None,None,None,None,None,None,None]

In [13]:
raw_svbps = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', '6*_id.nii.gz')))
labelvolumes = []
for j, labels_path in enumerate(raw_svbps):
    name = labels_path.split('/')[-1].split('_')[0]
    labels = nib.load(labels_path).get_fdata()
    labelvolume = []
    for i in range(1,11):
        labelvolume.append((labels==i).sum()*0.12**3)
    labelvolumes.append(labelvolume)

for i in range(0,9):
    volume_k = []
    for j in range(len(raw_svbps)):
        volume_k.append(labelvolumes[j][mapping_data['svbpnifti'][i]-1])
    raw_df['raw_volume_'+str(mapping_data['code'][i]).lower()][11:] = volume_k

/tmp/ipykernel_2563067/649017788.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['raw_volume_'+str(mapping_data['code'][i]).lower()][11:] = volume_k


In [14]:
print(raw_df)

     name  raw_volume_brain raw_volume_ctx raw_volume_hpf raw_volume_olf  \
0    M672        478.846080     136.489536      27.895104      26.160192   
1    M757        453.463488     130.045824      26.891136      25.164864   
2    M857        395.722368     112.067712      24.204096      24.129792   
3    M866        442.348992     126.105984       26.54208      22.572864   
4    M867        451.115136     129.613824      27.022464      27.689472   
5    M872        457.871616     131.227776      25.878528       21.75552   
6    M874        480.434112      136.27008      28.594944      27.298944   
7    M875        474.645312     134.198208      28.273536       26.04096   
8    M877        448.180992        129.384      25.949376      28.290816   
9    M886        493.680960     133.567488      29.562624       30.09312   
10  M8876        481.882176     136.346112      28.785024       30.87072   
11    653        486.974592     145.093248       26.10144       23.86368   
12    655   

# Volume ANTSMean

In [15]:
atlas_mask = os.path.join('dataset3', 'Atlas', 'P56_Annotation_128_norm_id_mask.nii.gz')
affine_transforms = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'MRI', '*.mat')))
inv_deform_transforms = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'MRI', '*_warp_inverse.nii.gz')))
for i in range(len(raw_mris)):
    output_name = raw_mris[i].split('/')[-1].split('.nii.gz')[0] + '_mask_ants.nii.gz'
    output = os.path.join('dataset3', 'GIN_Mean', 'Mask', output_name)
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', atlas_mask])
    command.extend(['-r', raw_mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'NearestNeighbor'])
    command.extend(['-t', '[' + affine_transforms[i] + ',1]'])
    command.extend(['-t', inv_deform_transforms[i]])
    subprocess.call(command) 

In [17]:
atlas_labels = os.path.join('dataset3', 'Atlas', 'atlas_gin_map6.nii.gz')
affine_transforms = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'MRI', '*.mat')))
inv_deform_transforms = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'MRI', '*_warp_inverse.nii.gz')))
for i in range(len(raw_mris)):
    output_name = raw_mris[i].split('/')[-1].split('.nii.gz')[0] + '_labels_ants.nii.gz'
    output = os.path.join('dataset3', 'GIN_Mean', 'Labels', output_name)
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', atlas_labels])
    command.extend(['-r', raw_mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'MultiLabel'])
    command.extend(['-t', '[' + affine_transforms[i] + ',1]'])
    command.extend(['-t', inv_deform_transforms[i]])
    subprocess.call(command) 

In [18]:
antsmean_df = pd.DataFrame()
mapping_csv = "dataset3/GIN/labels_mapping.csv"
mapping_data = pd.read_csv(mapping_csv)

In [19]:
mask_map6s = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'Mask', 'M*_mask_ants.nii.gz')))
mask_svbps = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'Mask', '6*_mask_ants.nii.gz')))
for mask_list in [mask_map6s, mask_svbps]:
    for mask_path in mask_list:
        name = mask_path.split('/')[-1].split('_')[0]
        mask = nib.load(mask_path).get_fdata()
        totalbrain = (mask==1).sum()*0.12**3   
        new_row = {'name':name, 'ants_volume_brain':totalbrain}
        antsmean_df = antsmean_df.append(new_row, ignore_index=True)

/tmp/ipykernel_2563067/707572551.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  antsmean_df = antsmean_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2563067/707572551.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  antsmean_df = antsmean_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2563067/707572551.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  antsmean_df = antsmean_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2563067/707572551.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  antsmean_df = antsmean_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2563067/707572551.py:9: FutureWarning: The fr

In [20]:
raw_map6s = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'Labels', 'M*_id_labels_ants.nii.gz')))
raw_svbps = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'Labels', '6*_id_labels_ants.nii.gz')))

labelvolumes_map6 = []
labelvolumes_svbp = []

for labels_list in [raw_map6s, raw_svbps]:            
    for j, labels_path in enumerate(labels_list):
        name = labels_path.split('/')[-1].split('_')[0]
        labels = nib.load(labels_path).get_fdata()
        labelvolume = []
        for i in range(1,11):
            labelvolume.append((labels==i).sum()*0.12**3)
        if labels_list == raw_map6s:
            labelvolumes_map6.append(labelvolume)
        else:            
            labelvolumes_svbp.append(labelvolume)

for i in range(0,10):
    volume_k = []
    for j in range(len(raw_map6s)):
        volume_k.append(labelvolumes_map6[j][mapping_data['map6nifti'][i]-1])
    for j in range(len(raw_svbps)):
        volume_k.append(labelvolumes_svbp[j][mapping_data['map6nifti'][i]-1])  
    antsmean_df['antsmean_volume_'+str(mapping_data['code'][i]).lower()] = volume_k

In [22]:
print(antsmean_df)
antsmean_df.to_csv("volume_antsmean.csv")

     name  ants_volume_brain  antsmean_volume_ctx  antsmean_volume_hpf  \
0    M672         467.679744           149.166144            29.341440   
1    M757         439.169472           137.495232            28.064448   
2    M857         397.671552           128.091456            25.266816   
3    M866         432.152064           134.761536            27.269568   
4    M867         446.413248           140.655744            28.104192   
5    M872         445.830912           140.657472            28.080000   
6    M874         466.900416           146.508480            29.871936   
7    M875         457.968384           147.381120            29.795904   
8    M877         441.586944           141.749568            27.264384   
9    M886         478.073664           149.261184            30.843072   
10  M8876         473.662080           149.259456            30.151872   
11    653         455.236416           145.720512            27.345600   
12    655         422.971200          

# Volume ANTS

In [140]:
atlas_mask = os.path.join('dataset3', 'Atlas', 'P56_Annotation_128_norm_id_mask.nii.gz')
affine_transforms = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*.mat')))
inv_deform_transforms = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*_warp_inverse.nii.gz')))
for i in range(len(raw_mris)):
    output_name = raw_mris[i].split('/')[-1].split('.nii.gz')[0] + '_mask_ants.nii.gz'
    output = os.path.join('dataset3', 'GIN', 'Mask', output_name)
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', atlas_mask])
    command.extend(['-r', raw_mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'NearestNeighbor'])
    command.extend(['-t', '[' + affine_transforms[i] + ',1]'])
    command.extend(['-t', inv_deform_transforms[i]])
    subprocess.call(command) 

NameError: name 'raw_mris' is not defined

In [141]:
atlas_labels = os.path.join('dataset3', 'Atlas', 'atlas_gin_map6.nii.gz')
affine_transforms = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*.mat')))
inv_deform_transforms = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*_warp_inverse.nii.gz')))
for i in range(len(raw_mris)):
    output_name = raw_mris[i].split('/')[-1].split('.nii.gz')[0] + '_labels_ants.nii.gz'
    output = os.path.join('dataset3', 'GIN', 'Labels', output_name)
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', atlas_labels])
    command.extend(['-r', raw_mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'MultiLabel'])
    command.extend(['-t', '[' + affine_transforms[i] + ',1]'])
    command.extend(['-t', inv_deform_transforms[i]])
    subprocess.call(command) 

NameError: name 'raw_mris' is not defined

In [143]:
ants_df = pd.DataFrame()
mapping_csv = "dataset3/GIN/labels_mapping.csv"
mapping_data = pd.read_csv(mapping_csv)

In [144]:
mask_map6s = sorted(glob(os.path.join('dataset3', 'GIN', 'Mask', 'M*_mask_ants.nii.gz')))
mask_svbps = sorted(glob(os.path.join('dataset3', 'GIN', 'Mask', '6*_mask_ants.nii.gz')))
for mask_list in [mask_map6s, mask_svbps]:
    for mask_path in mask_list:
        name = mask_path.split('/')[-1].split('_')[0]
        mask = nib.load(mask_path).get_fdata()
        totalbrain = (mask==1).sum()*0.12**3   
        new_row = {'name':name, 'ants_volume_brain':totalbrain}
        ants_df = ants_df.append(new_row, ignore_index=True)

/tmp/ipykernel_2165272/3786590276.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ants_df = ants_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2165272/3786590276.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ants_df = ants_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2165272/3786590276.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ants_df = ants_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2165272/3786590276.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ants_df = ants_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2165272/3786590276.py:9: FutureWarning: The frame.append method is deprec

In [145]:
raw_map6s = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', 'M*_id_labels_ants.nii.gz')))
raw_svbps = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', '6*_id_labels_ants.nii.gz')))

labelvolumes_map6 = []
labelvolumes_svbp = []

for labels_list in [raw_map6s, raw_svbps]:            
    for j, labels_path in enumerate(labels_list):
        name = labels_path.split('/')[-1].split('_')[0]
        labels = nib.load(labels_path).get_fdata()
        labelvolume = []
        for i in range(1,11):
            labelvolume.append((labels==i).sum()*0.12**3)
        if labels_list == raw_map6s:
            labelvolumes_map6.append(labelvolume)
        else:            
            labelvolumes_svbp.append(labelvolume)

for i in range(0,10):
    volume_k = []
    for j in range(len(raw_map6s)):
        volume_k.append(labelvolumes_map6[j][mapping_data['map6nifti'][i]-1])
    for j in range(len(raw_svbps)):
        volume_k.append(labelvolumes_svbp[j][mapping_data['map6nifti'][i]-1])  
    ants_df['ants_volume_'+str(mapping_data['code'][i]).lower()] = volume_k

In [146]:
print(ants_df)

     name  ants_volume_brain  ants_volume_ctx  ants_volume_hpf  \
0    M672         531.980352       165.298752        34.615296   
1    M757         489.341952       149.677632        32.598720   
2    M857         478.056384       143.752320        30.303936   
3    M866         490.948992       148.013568        31.368384   
4    M867         520.769088       153.786816        33.530112   
5    M872         504.418752       155.215872        32.562432   
6    M874         525.913344       162.380160        34.311168   
7    M875         519.915456       157.973760        34.003584   
8    M877         511.762752       154.732032        31.475520   
9    M886         550.575360       162.762048        35.135424   
10  M8876         541.619136       164.479680        35.655552   
11    653         524.587968       163.054080        33.156864   
12    655         499.860288       153.835200        33.397056   
13    656         487.741824       155.675520        29.987712   
14    660 

# Volume DL

In [80]:
atlas_mask = os.path.join('dataset3', 'Atlas', 'P56_Annotation_128_norm_id_mask.nii.gz')
affine_transforms = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*.mat')))
inv_deform_transforms = sorted(glob(os.path.join('output', 'GIN', 'baseline-sym-ddf', 'DeformableWarp', '*_invwarp.nii.gz')))
for i in range(len(raw_mris)):
    output_name = raw_mris[i].split('/')[-1].split('.nii.gz')[0] + '_mask_deep.nii.gz'
    output = os.path.join('dataset3', 'GIN', 'Mask', output_name)
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', atlas_mask])
    command.extend(['-r', raw_mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'NearestNeighbor'])
    command.extend(['-t', '[' + affine_transforms[i] + ',1]'])
    command.extend(['-t', inv_deform_transforms[i]])
    subprocess.call(command) 

In [81]:
atlas_labels = os.path.join('dataset3', 'Atlas', 'atlas_gin_map6.nii.gz')
affine_transforms = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*.mat')))
inv_deform_transforms = sorted(glob(os.path.join('output', 'GIN', 'baseline-sym-ddf', 'DeformableWarp', '*_invwarp.nii.gz')))
for i in range(len(raw_mris)):
    output_name = raw_mris[i].split('/')[-1].split('.nii.gz')[0] + '_labels_deep.nii.gz'
    output = os.path.join('dataset3', 'GIN', 'Labels', output_name)
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', atlas_labels])
    command.extend(['-r', raw_mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'MultiLabel'])
    command.extend(['-t', '[' + affine_transforms[i] + ',1]'])
    command.extend(['-t', inv_deform_transforms[i]])
    subprocess.call(command) 

In [147]:
deep_df = pd.DataFrame()
mapping_csv = "dataset3/GIN/labels_mapping.csv"
mapping_data = pd.read_csv(mapping_csv)

In [148]:
mask_map6s = sorted(glob(os.path.join('dataset3', 'GIN', 'Mask', 'M*_mask_deep.nii.gz')))
mask_svbps = sorted(glob(os.path.join('dataset3', 'GIN', 'Mask', '6*_mask_deep.nii.gz')))
for mask_list in [mask_map6s, mask_svbps]:
    for mask_path in mask_list:
        name = mask_path.split('/')[-1].split('_')[0]
        mask = nib.load(mask_path).get_fdata()
        totalbrain = (mask==1).sum()*0.12**3    
        new_row = {'name':name, 'deep_volume_brain':totalbrain}
        deep_df = deep_df.append(new_row, ignore_index=True)

/tmp/ipykernel_2165272/1637085885.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deep_df = deep_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2165272/1637085885.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deep_df = deep_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2165272/1637085885.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deep_df = deep_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2165272/1637085885.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deep_df = deep_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2165272/1637085885.py:9: FutureWarning: The frame.append method is deprec

In [149]:
raw_map6s = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', 'M*_id_labels_deep.nii.gz')))
raw_svbps = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', '6*_id_labels_deep.nii.gz')))

labelvolumes_map6 = []
labelvolumes_svbp = []

for labels_list in [raw_map6s, raw_svbps]:            
    for j, labels_path in enumerate(labels_list):
        name = labels_path.split('/')[-1].split('_')[0]
        labels = nib.load(labels_path).get_fdata()
        labelvolume = []
        for i in range(1,11):
            labelvolume.append((labels==i).sum()*0.12**3)
        if labels_list == raw_map6s:
            labelvolumes_map6.append(labelvolume)
        else:            
            labelvolumes_svbp.append(labelvolume)

for i in range(0,10):
    volume_k = []
    for j in range(len(raw_map6s)):
        volume_k.append(labelvolumes_map6[j][mapping_data['map6nifti'][i]-1])
    for j in range(len(raw_svbps)):
        volume_k.append(labelvolumes_svbp[j][mapping_data['map6nifti'][i]-1])  
    deep_df['deep_volume_'+str(mapping_data['code'][i]).lower()] = volume_k

# Volume Scenario1/2/3

In [4]:
for outfolder in ["scenario1", "scenario2", "scenario3"]:
    atlas_mask = os.path.join('dataset3', 'Atlas', 'P56_Annotation_128_norm_id_mask.nii.gz')
    affine_transforms = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*.mat')))
    inv_deform_transforms = sorted(glob(os.path.join('output', 'GIN', outfolder, 'DeformableWarp', '*_invwarp.nii.gz')))
    for i in range(len(raw_mris)):
        output_name = raw_mris[i].split('/')[-1].split('.nii.gz')[0] + '_mask_deep.nii.gz'
        output = os.path.join('dataset3', 'GIN', 'Mask', output_name)
        command = ['antsApplyTransforms']
        command.extend(['-d', '3'])
        command.extend(['-i', atlas_mask])
        command.extend(['-r', raw_mris[i]])
        command.extend(['-o', output])
        command.extend(['-n', 'NearestNeighbor'])
        command.extend(['-t', '[' + affine_transforms[i] + ',1]'])
        command.extend(['-t', inv_deform_transforms[i]])
        subprocess.call(command) 

    atlas_labels = os.path.join('dataset3', 'Atlas', 'atlas_gin_map6.nii.gz')
    affine_transforms = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*.mat')))
    inv_deform_transforms = sorted(glob(os.path.join('output', 'GIN', outfolder, 'DeformableWarp', '*_invwarp.nii.gz')))
    for i in range(len(raw_mris)):
        output_name = raw_mris[i].split('/')[-1].split('.nii.gz')[0] + '_labels_deep.nii.gz'
        output = os.path.join('dataset3', 'GIN', 'Labels', output_name)
        command = ['antsApplyTransforms']
        command.extend(['-d', '3'])
        command.extend(['-i', atlas_labels])
        command.extend(['-r', raw_mris[i]])
        command.extend(['-o', output])
        command.extend(['-n', 'MultiLabel'])
        command.extend(['-t', '[' + affine_transforms[i] + ',1]'])
        command.extend(['-t', inv_deform_transforms[i]])
        subprocess.call(command) 

    deep_df = pd.DataFrame()
    mapping_csv = "dataset3/GIN/labels_mapping.csv"
    mapping_data = pd.read_csv(mapping_csv)

    mask_map6s = sorted(glob(os.path.join('dataset3', 'GIN', 'Mask', 'M*_mask_deep.nii.gz')))
    mask_svbps = sorted(glob(os.path.join('dataset3', 'GIN', 'Mask', '6*_mask_deep.nii.gz')))
    for mask_list in [mask_map6s, mask_svbps]:
        for mask_path in mask_list:
            name = mask_path.split('/')[-1].split('_')[0]
            mask = nib.load(mask_path).get_fdata()
            totalbrain = (mask==1).sum()*0.12**3    
            new_row = {'name':name, 'deep_volume_brain':totalbrain}
            deep_df = deep_df.append(new_row, ignore_index=True)

    raw_map6s = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', 'M*_id_labels_deep.nii.gz')))
    raw_svbps = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', '6*_id_labels_deep.nii.gz')))

    labelvolumes_map6 = []
    labelvolumes_svbp = []

    for labels_list in [raw_map6s, raw_svbps]:            
        for j, labels_path in enumerate(labels_list):
            name = labels_path.split('/')[-1].split('_')[0]
            labels = nib.load(labels_path).get_fdata()
            labelvolume = []
            for i in range(1,11):
                labelvolume.append((labels==i).sum()*0.12**3)
            if labels_list == raw_map6s:
                labelvolumes_map6.append(labelvolume)
            else:            
                labelvolumes_svbp.append(labelvolume)

    for i in range(0,10):
        volume_k = []
        for j in range(len(raw_map6s)):
            volume_k.append(labelvolumes_map6[j][mapping_data['map6nifti'][i]-1])
        for j in range(len(raw_svbps)):
            volume_k.append(labelvolumes_svbp[j][mapping_data['map6nifti'][i]-1])  
        deep_df['deep_volume_'+str(mapping_data['code'][i]).lower()] = volume_k
    
    deep_df.to_csv("volume_deep_" + str(outfolder) + ".csv")

/tmp/ipykernel_2568358/2634440040.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deep_df = deep_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2568358/2634440040.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deep_df = deep_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2568358/2634440040.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deep_df = deep_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2568358/2634440040.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deep_df = deep_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2568358/2634440040.py:46: FutureWarning: The frame.append method is d

/tmp/ipykernel_2568358/2634440040.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deep_df = deep_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2568358/2634440040.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deep_df = deep_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2568358/2634440040.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deep_df = deep_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2568358/2634440040.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deep_df = deep_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2568358/2634440040.py:46: FutureWarning: The frame.append method is d

/tmp/ipykernel_2568358/2634440040.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deep_df = deep_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2568358/2634440040.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deep_df = deep_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2568358/2634440040.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deep_df = deep_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2568358/2634440040.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  deep_df = deep_df.append(new_row, ignore_index=True)
/tmp/ipykernel_2568358/2634440040.py:46: FutureWarning: The frame.append method is d

# Save

In [150]:
genotypes = [
    'WT',
    'Het',
    'Het',
    'Het',
    'Het',
    'WT',
    'WT',
    'WT',
    'Het',
    'WT',
    'WT',
    'WT',
    'KO',
    'KO',
    'WT',
    'KO',
    'WT',
    'WT',
    'WT',
    'KO',
    'KO',
    'WT',
    'KO',
    'KO',
    'WT',
    'KO',
    'WT',]
study = []
for i in range(11):
    study.append('MAP6')
for i in range(16):
    study.append('SVBP')
print(study)
for df in [raw_df, ants_df, deep_df]:
    df['genotype'] = genotypes
    df['study'] = study
raw_df.to_csv ("volume_raw.csv")
ants_df.to_csv("volume_ants.csv")
deep_df.to_csv("volume_deep.csv")

['MAP6', 'MAP6', 'MAP6', 'MAP6', 'MAP6', 'MAP6', 'MAP6', 'MAP6', 'MAP6', 'MAP6', 'MAP6', 'SVBP', 'SVBP', 'SVBP', 'SVBP', 'SVBP', 'SVBP', 'SVBP', 'SVBP', 'SVBP', 'SVBP', 'SVBP', 'SVBP', 'SVBP', 'SVBP', 'SVBP', 'SVBP']


# Test

In [152]:
raw_mris = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*_id.nii.gz')))
atlas_labels = os.path.join('dataset3', 'Atlas', 'P56_Annotation_128_norm_id.nii.gz')
affine_transforms = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*.mat')))
inv_deform_transforms = sorted(glob(os.path.join('output', 'GIN', 'baseline-sym-ddf', 'DeformableWarp', '*_invwarp.nii.gz')))
for i in range(len(raw_mris)):
    output_name = raw_mris[i].split('/')[-1].split('.nii.gz')[0] + '_atlas_deep.nii.gz'
    output = os.path.join('dataset3', 'GIN', 'Labels', output_name)
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', atlas_labels])
    command.extend(['-r', raw_mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'MultiLabel'])
    command.extend(['-t', '[' + affine_transforms[i] + ',1]'])
    command.extend(['-t', inv_deform_transforms[i]])
    subprocess.call(command) 

In [153]:
raw_mris = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*_id.nii.gz')))
atlas_labels = os.path.join('dataset3', 'Atlas', 'P56_Annotation_128_norm_id.nii.gz')
affine_transforms = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*.mat')))
inv_deform_transforms = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*_warp_inverse.nii.gz')))
for i in range(len(raw_mris)):
    output_name = raw_mris[i].split('/')[-1].split('.nii.gz')[0] + '_atlas_ants.nii.gz'
    output = os.path.join('dataset3', 'GIN', 'Labels', output_name)
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', atlas_labels])
    command.extend(['-r', raw_mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'MultiLabel'])
    command.extend(['-t', '[' + affine_transforms[i] + ',1]'])
    command.extend(['-t', inv_deform_transforms[i]])
    subprocess.call(command) 

In [155]:
def query_data_by_name(data, name):
    list = []
    for i in range(len(data)):
        if data['acronym'][i] == name:
            j = i 
            id = data['id'][j]
    id = "/" + str(id) + "/"
    for i in range(len(data)):
        if id in data['structure_id_path'][i]:
            list.append(data['graph_order'][i])
    return list   

In [189]:
query_csv = "dataset3/Atlas/query.csv"
query_data = pd.read_csv(query_csv )
mapping_csv = "dataset3/GIN/labels_mapping.csv"
mapping_data = pd.read_csv(mapping_csv)

In [190]:
print(mapping_data)

                    name     code     map6tif  map6nifti        svbptif  \
0                 cortex      CTX      yellow          5      darkgreen   
1  hippocampal formation      HPF   darkgreen          3            idk   
2         olfactory bulb      OLF  lightgreen          9     lightgreen   
3               striatum      STR    darkblue          4      lightblue   
4        globus pallidus  GPe/GPi        pink          6  darkgreenblue   
5              cerebelum       CB      salmon          8         yellow   
6               coliculi    MBsen         red          2           pink   
7               thalamus       TH   lightblue          7         salmon   
8           hypothalamus       HY      purple          1            red   
9              brainstem       BS      orange         10           none   

   svbpnifti           atlasnifti  Unnamed: 7  
0          2                    3         NaN  
1          3                  454         NaN  
2          1              379;

In [207]:
atlas_antsx = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', '*_atlas_ants.nii.gz')))
atlas_deepx = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', '*_atlas_deep.nii.gz')))
for j in range(len(atlas_antsx)):
    name = atlas_antsx[j].split('/')[-1].split('_')[0]
    volumes = [name]
    codes = ['name']
    for atlas_x in [atlas_antsx[j], atlas_deepx[j]]:
        atlas = nib.load(atlas_x).get_fdata()
        for i in range(len(mapping_data)):
            code_list = mapping_data['code'][i]
            if atlas_x == atlas_antsx[j]:
                codes.append(code_list + '_ants')
            else:
                codes.append(code_list + '_deep')
            label_list = []
            for code in code_list.split('/'):
                liste = query_data_by_name(query_data, code)
                for label in liste:
                    label_list.append(label)
            x = np.isin(atlas, label_list).sum()*0.12**3
            volumes.append(x)
    if j == 0:
        atlas_df = pd.DataFrame(columns=codes)
    atlas_df.loc[len(atlas_df)] = volumes    

     name    CTX_ants   HPF_ants   OLF_ants   STR_ants  GPe/GPi_ants  \
0     653  234.439488  50.108544  63.835776  48.885120      1.854144   
1     655  221.965056  50.542272  60.046272  44.409600      1.759104   
2     656  224.135424  47.468160  63.199872  44.216064      1.594944   
3     660  220.828032  49.488192  53.025408  45.817920      1.683072   
4     661  226.532160  46.645632  63.811584  47.205504      1.508544   
5     662  235.927296  50.426496  67.853376  47.678976      1.852416   
6     663  218.002752  50.820480  47.623680  43.787520      1.741824   
7     664  223.224768  47.729088  61.872768  43.521408      1.721088   
8     665  213.886656  46.177344  55.219968  42.754176      1.472256   
9     667  208.954944  44.247168  58.155840  42.040512      1.406592   
10    668  226.665216  48.606912  61.354368  48.014208      1.760832   
11    670  216.983232  47.074176  60.455808  43.974144      1.755648   
12    675  208.573056  42.868224  59.047488  42.488064      1.44

In [208]:
genotypes = [
    'WT',
    'KO',
    'KO',
    'WT',
    'KO',
    'WT',
    'WT',
    'WT',
    'KO',
    'KO',
    'WT',
    'KO',
    'KO',
    'WT',
    'KO',
    'WT',
    'WT',
    'Het',
    'Het',
    'Het',
    'Het',
    'WT',
    'WT',
    'WT',
    'Het',
    'WT',
    'WT',    
    ]
study = []
for i in range(16):
    study.append('SVBP')
for i in range(11):
    study.append('MAP6')
atlas_df['genotype'] = genotypes
atlas_df['study'] = study
atlas_df.to_csv("volume_atlas.csv")
print(atlas_df)

     name    CTX_ants   HPF_ants   OLF_ants   STR_ants  GPe/GPi_ants  \
0     653  234.439488  50.108544  63.835776  48.885120      1.854144   
1     655  221.965056  50.542272  60.046272  44.409600      1.759104   
2     656  224.135424  47.468160  63.199872  44.216064      1.594944   
3     660  220.828032  49.488192  53.025408  45.817920      1.683072   
4     661  226.532160  46.645632  63.811584  47.205504      1.508544   
5     662  235.927296  50.426496  67.853376  47.678976      1.852416   
6     663  218.002752  50.820480  47.623680  43.787520      1.741824   
7     664  223.224768  47.729088  61.872768  43.521408      1.721088   
8     665  213.886656  46.177344  55.219968  42.754176      1.472256   
9     667  208.954944  44.247168  58.155840  42.040512      1.406592   
10    668  226.665216  48.606912  61.354368  48.014208      1.760832   
11    670  216.983232  47.074176  60.455808  43.974144      1.755648   
12    675  208.573056  42.868224  59.047488  42.488064      1.44

In [ ]:
atlas_mask = os.path.join('dataset3', 'Atlas', 'P56_Annotation_128_norm_id_mask.nii.gz')
affine_transforms = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', '*.mat')))
inv_deform_transforms = sorted(glob(os.path.join('output', 'GIN', 'baseline-sym-ddf', 'DeformableWarp', '*_invwarp.nii.gz')))
for i in range(len(raw_mris)):
    output_name = raw_mris[i].split('/')[-1].split('.nii.gz')[0] + '_mask_deep.nii.gz'
    output = os.path.join('dataset3', 'GIN', 'Mask', output_name)
    command = ['antsApplyTransforms']
    command.extend(['-d', '3'])
    command.extend(['-i', atlas_mask])
    command.extend(['-r', raw_mris[i]])
    command.extend(['-o', output])
    command.extend(['-n', 'NearestNeighbor'])
    command.extend(['-t', '[' + affine_transforms[i] + ',1]'])
    command.extend(['-t', inv_deform_transforms[i]])
    subprocess.call(command) 
    break